In [1]:
import numpy as np

In [2]:
# useful matrices
Identity_22 = np.eye(2, dtype=np.complex128)
Pauli_x = np.array([[0, 1], [1, 0]], dtype=np.complex128)

# threshold
thr = 10**-9

In [7]:
def is_unitary(A):
    n = A.shape[0]
    if (A.shape != (n, n)):
        raise ValueError("Matrix is not square.")
    A = np.array(A)
    return np.allclose(np.eye(n), A @ A.conj().T)


def is_identity(A):
    n = A.shape[0]
    if (A.shape != (n, n)):
        raise ValueError("Matrix is not square.")
    return np.allclose(A, np.eye(n))


def elimination_matrix(a,b):
    # a, b allowed to be complex
    
    # impose theta real + positive {eq.10}
    theta = np.arctan(abs(b/a))
    
    # lambda is the negative arg() of a
    lamda = - np.angle(a)
    
    # {eq.12}
    mu = np.pi + np.angle(b)
    
    # {eq.7}
    U_special = np.array([ [np.exp(1j*lamda) * np.cos(theta), np.exp(1j*mu) * np.sin(theta)],
                           [-np.exp(-1j*mu) * np.sin(theta), np.exp(-1j*lamda) * np.cos(theta)] ])
    
    return U_special


def two_level_decomp(A):
    
    """
    decomp - list of 2x2 unitaries that decompose A. To reconstruct A, the list of unitaries
             in decomp must be reversed
             
    indices - list of non-trivial rows on which matrices in decomp act. For example, [1,3] indicates
              that row 1 and 3 are non-trivial; all other elements are 1.
    """
    
    n = A.shape[0]
    decomp = []
    indices = []
    A_c = np.copy(A)

    for i in range(n-2):
        for j in range(n-1, i, -1):

            a = A_c[i,j-1]
            b = A_c[i,j]

            # --- need checks --- 
            # if A[i,j] = 0, nothing to do! Except in last row - need to check diagonal element is 1 
            if abs(A_c[i,j]) < thr:
                U_22 = Identity_22

                if j == i+1:
                    U_22 = np.array([[1 / a, 0], [0, a]])

            # if A[i,j-1] = 0, need to swap columns - again checking last row to ensure diagonal element is 1 
            elif abs(A_c[i,j-1]) < thr:
                U_22 = Pauli_x

                if j == i+1:
                    U_22 = np.array([[1 / b, 0], [0, b]])

            # Special unitary matrix
            else: 
                U_22 = elimination_matrix(a,b)

            # ----- U_22 found -----

            # multiply submatrix of A with U_22
            A_c[:,(j-1,j)] = A_c[:,(j-1,j)] @ U_22

            # If not the identity matrix - represents a gate! So should store
            if not is_identity(U_22):
                decomp.append(U_22.conj().T)
                indices.append(np.array([j-1,j]))


        # check for diagonal element equal to 1
        assert np.allclose(A_c[i,i],1.0)
    
    # lower right hand 2x2 matrix remaining after decomp
    lower_rh_matrix = A_c[n-2:n, n-2:n]
    
    # if not equal to I - is a non trivial gate
    if not is_identity(lower_rh_matrix):
        decomp.append(lower_rh_matrix)
        indices.append(np.array([n-2,n-1]))

    return decomp, indices


def gray_method(A):
    
    n = A.shape[0]
    M = np.copy(A)

    # using bitwise_xor find Gray permutations
    permutations = []
    for i in range(n):
        permutations.append(i ^ (i // 2))
        
    # 
    M[:,:] = M[:,permutations]
    M[:,:] = M[permutations,:]
    
    decomp, indices = two_level_decomp(M)
    new_decomp = []
    new_ind = []

    for i in range(len(indices)):
        
        t = np.take(permutations, indices[i])
        if t[0]>t[1]:
            new_decomp.append(decomp[i].T)
            new_ind.append(np.sort(t))

        else:
            new_decomp.append(decomp[i])
            new_ind.append(t)
            
    return new_decomp, new_ind

In [77]:
from functools import reduce
def recompose_matrix(matrix_seq, location_seq, num_qubits):
    U_list = []
    for M, loc in zip(matrix_seq, location_seq):
        
        U = np.eye(2**num_qubits, dtype=np.complex128)
        r1 = loc[0]
        
        U[r1:r1+2,r1:r1+2] = M
        U_list.append(U)

    U_list.reverse()

    A_approx = reduce(lambda a,b : a @ b, U_list)
    
    return A_approx

In [5]:
# test matrix
w = np.exp((2j / 3) * np.pi)
A = np.array([[1, 1, 1, 0], 
                  [1, w, w * w, 0],
                  [1, w * w, w, 0], 
                  [0, 0, 0, -1j*np.sqrt(3)]]) / np.sqrt(3)

In [6]:
# from scipy.stats import unitary_group
# nq = 2
# A = unitary_group.rvs(2**nq)
# decomp, indices = two_level_decomp(A)
# decomp, indices

# Stress tests

In [54]:
A = np.array([[1, 1, 1, 0], 
              [1, w, w * w, 0],
              [1, w * w, w, 0], 
              [0, 0, 0, -1j*np.sqrt(3)]]) / np.sqrt(3)

B = unitary_group.rvs(2**2)

num_qubits_C = 8

C = unitary_group.rvs(2**num_qubits_C)


NameError: name 'unitary_group' is not defined

## Two-level-decomp Method

In [76]:
# Test on A
import numpy as np
from scipy.linalg import expm, sinm, cosm

nqbits = 4

m = np.random.rand(2**nqbits,2**nqbits)
H = m + np.conj(np.transpose(m))
u = expm(1.j*H)

matrix_seq, location_seq = two_level_decomp(u)
A_approx = recompose_matrix(matrix_seq, location_seq, nqbits)
print(np.allclose(u, A_approx))

[14 15]
[13 14]
[12 13]
[11 12]
[10 11]
[ 9 10]
[8 9]
[7 8]
[6 7]
[5 6]
[4 5]
[3 4]
[2 3]
[1 2]
[0 1]
[14 15]
[13 14]
[12 13]
[11 12]
[10 11]
[ 9 10]
[8 9]
[7 8]
[6 7]
[5 6]
[4 5]
[3 4]
[2 3]
[1 2]
[14 15]
[13 14]
[12 13]
[11 12]
[10 11]
[ 9 10]
[8 9]
[7 8]
[6 7]
[5 6]
[4 5]
[3 4]
[2 3]
[14 15]
[13 14]
[12 13]
[11 12]
[10 11]
[ 9 10]
[8 9]
[7 8]
[6 7]
[5 6]
[4 5]
[3 4]
[14 15]
[13 14]
[12 13]
[11 12]
[10 11]
[ 9 10]
[8 9]
[7 8]
[6 7]
[5 6]
[4 5]
[14 15]
[13 14]
[12 13]
[11 12]
[10 11]
[ 9 10]
[8 9]
[7 8]
[6 7]
[5 6]
[14 15]
[13 14]
[12 13]
[11 12]
[10 11]
[ 9 10]
[8 9]
[7 8]
[6 7]
[14 15]
[13 14]
[12 13]
[11 12]
[10 11]
[ 9 10]
[8 9]
[7 8]
[14 15]
[13 14]
[12 13]
[11 12]
[10 11]
[ 9 10]
[8 9]
[14 15]
[13 14]
[12 13]
[11 12]
[10 11]
[ 9 10]
[14 15]
[13 14]
[12 13]
[11 12]
[10 11]
[14 15]
[13 14]
[12 13]
[11 12]
[14 15]
[13 14]
[12 13]
[14 15]
[13 14]
[14 15]
True


In [65]:
# Test on B
matrix_seq, location_seq = two_level_decomp(B)
B_approx = recompose_matrix(matrix_seq, location_seq, num_qubits=2)
np.allclose(B, B_approx)

NameError: name 'B' is not defined

In [212]:
# Test on C
matrix_seq, location_seq = two_level_decomp(C)
C_approx = recompose_matrix(matrix_seq, location_seq, num_qubits=num_qubits_C)
np.allclose(C, C_approx)

True

## Gray's Method

In [218]:
# Test on A
matrix_seq, location_seq = gray_method(A)
A_approx = recompose_matrix(matrix_seq, location_seq, num_qubits=2)
np.allclose(A, A_approx)

True

In [220]:
# Test on B
matrix_seq, location_seq = gray_method(B)
B_approx = recompose_matrix(matrix_seq, location_seq, num_qubits=2)
np.allclose(B, B_approx)

# Test on C
matrix_seq, location_seq = gray_method(C)
C_approx = recompose_matrix(matrix_seq, location_seq, num_qubits=num_qubits_C)
np.allclose(C, C_approx)

KeyboardInterrupt: 

In [6]:
w

(-0.4999999999999998+0.8660254037844388j)

In [8]:
c,d = two_level_decomp(A)

In [9]:
c

[array([[ 0.70710678-0.00000000e+00j,  0.70710678-8.65956056e-17j],
        [-0.70710678-8.65956056e-17j,  0.70710678-0.00000000e+00j]]),
 array([[ 0.57735027-0.00000000e+00j,  0.81649658-9.99919924e-17j],
        [-0.81649658-9.99919924e-17j,  0.57735027-0.00000000e+00j]]),
 array([[-7.07106781e-01+8.65956056e-17j, -5.14325540e-16-7.07106781e-01j],
        [ 5.14325540e-16-7.07106781e-01j, -7.07106781e-01-8.65956056e-17j]]),
 array([[-6.98727119e-16-1.j,  0.00000000e+00+0.j],
        [ 0.00000000e+00+0.j,  0.00000000e+00-1.j]])]

In [10]:
d

[array([1, 2]), array([0, 1]), array([1, 2]), array([2, 3])]

In [20]:
 tol = 1e-14
>>> c[0].real[abs(c[0].real) < tol] = 0.0
>>> c[0].imag[abs(c[0].imag) < tol] = 0.0

In [21]:
c[0]

array([[ 0.70710678+0.j,  0.70710678+0.j],
       [-0.70710678+0.j,  0.70710678+0.j]])

In [23]:
len(c)

4

In [24]:
c

[array([[ 0.70710678+0.j,  0.70710678+0.j],
        [-0.70710678+0.j,  0.70710678+0.j]]),
 array([[ 0.57735027-0.00000000e+00j,  0.81649658-9.99919924e-17j],
        [-0.81649658-9.99919924e-17j,  0.57735027-0.00000000e+00j]]),
 array([[-7.07106781e-01+8.65956056e-17j, -5.14325540e-16-7.07106781e-01j],
        [ 5.14325540e-16-7.07106781e-01j, -7.07106781e-01-8.65956056e-17j]]),
 array([[-6.98727119e-16-1.j,  0.00000000e+00+0.j],
        [ 0.00000000e+00+0.j,  0.00000000e+00-1.j]])]